##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [2]:
%pip install -q -U "google-generativeai>=0.7.2"

In [3]:
%pip install -q wikipedia

  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [4]:
import json
import textwrap

import google.generativeai as genai

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [7]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [8]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-2.0-flash')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [9]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Vibe coding', 'DeepSeek']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Large language model did not match with any page id, hence skipping.
Fetching page: "Vibe coding"
Information Source: https://en.wikipedia.org/wiki/Vibe_coding
Fetching page: "DeepSeek"
Information Source: https://en.wikipedia.org/wiki/DeepSeek
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Vibe_coding
     https://en.wikipedia.org/wiki/DeepSeek


Here is what the search results look like:

In [10]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> LLMs are large language models tuned for coding, which are used in vibe coding to generate software from natural language prompts.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/Vibe_coding

> LLMs are large language models. DeepSeek, a Chinese artificial intelligence company, develops LLMs. DeepSeek-R1 provides responses comparable to other contemporary large language models, such as OpenAI's GPT-4 and o1. Its training cost is reported to be significantly lower than other LLMs.
> 
> 
> Based on:
>   https://en.wikipedia.org/wiki/DeepSeek

### Pass the tools to the model

If you pass a list of functions to the `GenerativeModel`'s `tools` argument,
it will extract a schema from the function's signature and type hints, and then pass schema along to the API calls. In response the model may return a `FunctionCall` object asking to call the function.

Note: This approach only handles annotations of `AllowedTypes = int | float | str | dict | list['AllowedTypes']`

The `GenerativeModel` will keep a reference to the function inself, so that it _can_ execute the function locally later.

In [11]:
model = genai.GenerativeModel(
    'gemini-2.0-flash',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [12]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [13]:
model = genai.GenerativeModel(
    'gemini-2.0-flash', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "deep sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep sea ecosystem survival strategies"
Related search terms: ['Kelp forest', 'Sea lion', 'Marine coastal ecosystem']
Fetching page: "Kelp forest"
Information Source: https://en.wikipedia.org/wiki/Kelp_forest
Fetching page: "Sea lion"
Information Source: https://en.wikipedia.org/wiki/Sea_lion
Fetching page: "Marine coastal ecosystem"
Information Source: https://en.wikipedia.org/wiki/Marine_coastal_ecosystem
Searching for "hydrothermal vent life"
Related search terms: ['Hydrothermal vent', 'Endeavour Hydrothermal Vent

In [14]:
to_markdown(res.text)

> Deep-sea life survives in a challenging environment characterized by low temperatures, darkness, high pressure, and scarce food. Organisms have adapted in various ways to overcome these challenges:
> 
> 1.  **Adaptations to Darkness:** Many deep-sea creatures have developed large, tubular eyes with only rod cells, an upward field of vision, lateral compression of the body, counter-illumination via bioluminescence and retroreflectors behind the retina.
> 2.  **Feeding Adaptations:** Deep-sea life relies on scavenging, predation, filtration, and marine snow (organic material falling from upper waters). Filter feeders use tentacles to feed upon organic particles.
> 3.  **General Biological Adaptations:** Many species have jelly-like flesh for buoyancy, small size, slow metabolisms, unspecialized diets, elongated bodies with weak muscles, extendable jaws, and hermaphroditism.
> 4.  **Adaptation to Hydrostatic Pressure:** Deep-sea organisms have changes in protein structure and function, increased salt bridges in actin, accumulation of specific osmolytes like Trimethylamine N-oxide (TMAO) to protect proteins.
> 5.  **Chemosynthesis-based Ecosystems:** Hydrothermal vent communities rely on chemosynthetic bacteria instead of sunlight. Tube worms (Riftia) have a symbiotic relationship with chemosynthetic bacteria.
> 6. **Deep-sea gigantism:** Some deep-sea animals tend to be larger than their shallow-water relatives as an adaptation to colder temperatures, food scarcity, reduced predation pressure, and increased dissolved oxygen concentrations.


Check for additional citations:

In [15]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [16]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

In the chat history you can see all 4 steps:

1. The user sent the query.
2. The model replied with a `genai.protos.FunctionCall` calling the `wikipedia_search` with a number of relevant searches.
3. Because you set `enable_automatic_function_calling=True` when creating the `genai.ChatSession`, it  executed the search function and returned the list of article summaries to the model.
4. Folliwing the instructions in the prompt, the model generated a final answer based on those summaries.


## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [17]:
chat = model.start_chat()

In [18]:
result = chat.send_message(instructions.format(query=query))

Initially the model returns a FunctionCall:

In [19]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "deep-sea life adaptations",
      "deep-sea ecosystems",
      "hydrothermal vent ecosystems",
      "deep-sea chemosynthesis",
      "how do deep-sea creatures get energy?",
      "deep-sea food web",
      "challenges of deep-sea life",
      "deep-sea pressure adaptations",
      "deep-sea temperature adaptations",
      "deep-sea oxygen levels",
      "bioluminescence in deep-sea life"
    ]
  },
  "id": ""
}


In [20]:
fc['name']

'wikipedia_search'

Call the function with generated arguments to get the results.

In [21]:
summaries = wikipedia_search(**fc['args'])

Searching for "deep-sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Deep sea did not match with any page id, hence skipping.
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep-sea ecosystems"
Related search terms: ['Deep-sea community', 'Deep sea mining', 'Deep sea']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Deep-sea community did not match with any page id, hence skipping.
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Deep sea mining did not match with any page id, hence skipping.
Searching for "hydrothermal vent 

Now send the `FunctionResult` to the model.

In [22]:
response = chat.send_message(
    genai.protos.Content(
      parts=[genai.protos.Part(
          function_response = genai.protos.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> Deep-sea life survives in a challenging environment through various adaptations. Here's a breakdown:
> 
> *   **Gigantism:** Deep-sea animals tend to be larger than their shallow-water relatives, potentially due to colder temperatures, food scarcity, reduced predation, and increased dissolved oxygen.
> *   **Food Scarcity:** Some creatures, like giant isopods, gorge on food when available and can survive for extended periods without it.
> *   **Hydrothermal Vent Ecosystems:** Some deep-sea life thrives around hydrothermal vents, which provide energy through chemosynthesis. These vents release fluids rich in volatile gases like hydrogen sulfide, hydrogen, and methane. Bacteria mats around the vents, including Epsilonproteobacteria and Gammaproteobacteria, oxidize hydrogen and hydrogen sulfide.
> *   **Deep-Sea Food Web:** The ping pong tree sponge captures small crustaceans and other invertebrates. It also provides shelter for other deep-sea species.

## Re-ranking the search results

Helper function to embed the content:

In [23]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [24]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [25]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [26]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [27]:
print(summaries[np.argmax(sim_value)])

Here's the extracted information about deep-sea life adaptations from the provided source:

*   **Deep-sea gigantism (abyssal gigantism):** The tendency for deep-sea dwelling animals to be larger than their shallower-water relatives.
*   **Proposed explanations for gigantism:**
    *   Adaptation to colder temperature
    *   Food scarcity
    *   Reduced predation pressure
    *   Increased dissolved oxygen concentrations
*   **Food Scarcity adaptation:** Giant isopods gorge on food when available and can survive 5 years without food in captivity.
*   **Increased dissolved oxygen:** Larger organisms are able to intake more dissolved oxygen.

Based on:
  https://en.wikipedia.org/wiki/Deep-sea_gigantism


### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [28]:
hypothetical_ans_model = genai.GenerativeModel('gemini-2.0-flash')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> Okay, imagine the deep sea as this completely alien planet, right? It's nothing like the surface. We're talking crushing pressures, like having a [GiantPlaceholderNumber] elephants standing on your head! And absolute darkness, colder than a [SpecificGeographicLocationPlaceholder] winter. So how do anything survive down there?
> 
> Well, it's all about adaptation and a really, really robust ecosystem built on some pretty unusual energy sources.
> 
> First off, those pressures. You'd think they'd just implode everything, but deep-sea creatures have evolved some clever ways around it. Their bodies are often lacking air-filled cavities like lungs or swim bladders. Instead, they tend to be built with a high water content and unique cell structures that are incredibly resistant to compression. Think of them less like squishy beach balls and more like… well, a super dense [RandomObjectPlaceholder].
> 
> Then there's the darkness. Photosynthesis? Forget about it! No sunlight means no plants. Instead, deep-sea ecosystems rely heavily on "marine snow" - which is basically organic debris raining down from the sunlit surface waters. Imagine a constant, albeit sparse, snowfall of dead plankton, fish poop, and other goodies. This is the base of the food chain for many creatures. Things like [PlaceholderAnimalName], which is a detritivore, filter-feed on this stuff.
> 
> But that's not the *only* energy source. Around hydrothermal vents, volcanic hotspots on the ocean floor, things get really interesting. These vents spew out chemicals like hydrogen sulfide and methane. Bacteria and archaea, some incredibly hardy single-celled organisms, are able to chemosynthesize – basically, they create energy from these chemicals, just like plants create energy from the sun. These chemosynthetic bacteria form the base of a whole new food chain around the vents. So you get things like tube worms, which have no mouth or gut, relying entirely on symbiotic bacteria living inside them. It's like a little [AbstractConceptPlaceholder] factory in their bodies!
> 
> So, you have creatures adapted to the pressure, feeding on marine snow or chemosynthetic bacteria, and you have incredible food webs built upon these adaptations. And let's not forget bioluminescence! Many deep-sea creatures can create their own light. This isn't just for show; it's used for attracting prey, communicating, camouflage, and even warding off predators. Think of the anglerfish with its glowing lure – that's a classic example of bioluminescence in action.
> 
> Basically, deep-sea life is a testament to the power of evolution and adaptation. It's a bizarre, beautiful, and incredibly resilient ecosystem operating under conditions that would be utterly inhospitable to most life on Earth. It's a reminder that life finds a way, even in the most extreme environments, by being a real [FunnyAdjectivePlaceholder] ecosystem.


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [29]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [30]:
sim_value = dot_product(search_res, hypothetical_ans)

In [31]:
sim_value

array([[0.76211845],
       [0.646311  ],
       [0.73430291],
       [0.71097457],
       [0.73318881],
       [0.69100806]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [32]:
to_markdown(summaries[np.argmax(sim_value)])

> Here's the extracted information about deep-sea life adaptations from the provided source:
> 
> *   **Deep-sea gigantism (abyssal gigantism):** The tendency for deep-sea dwelling animals to be larger than their shallower-water relatives.
> *   **Proposed explanations for gigantism:**
>     *   Adaptation to colder temperature
>     *   Food scarcity
>     *   Reduced predation pressure
>     *   Increased dissolved oxygen concentrations
> *   **Food Scarcity adaptation:** Giant isopods gorge on food when available and can survive 5 years without food in captivity.
> *   **Increased dissolved oxygen:** Larger organisms are able to intake more dissolved oxygen.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Deep-sea_gigantism

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.